# 12. Newton and Quasi-Newton

## Daniel Dimitrov

In [1]:
import numpy as np
from numpy import linalg as la
import matplotlib.pyplot as plt
from scipy import optimize as opt
from scipy import sparse

In [2]:
def newton(df,d2f,x0,maxiters):
    
    err = 100
    tol = 1e-9
    counter = 0
    x = x0
    converged = False
    
    while (err > tol) and (counter+1 < maxiters ):
        x_new = x - la.inv(d2f(x))@df(x)
        err = la.norm(x-x_new, np.inf)
        x = x_new
        counter +=1
    if counter < maxiters:
        converged = True
    return x, err, converged

In [3]:
f = opt.rosen
df = opt.rosen_der
d2f = opt.rosen_hess
x0=[-2,2]
# opt's solution 
opt.fmin_bfgs(f=f, x0=[-2,2], fprime=df, maxiter=50)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 35
         Function evaluations: 42
         Gradient evaluations: 42


array([1.00000021, 1.00000045])

In [4]:
#my solution
x, err, converged = newton(df,d2f,x0,100)
print(x, err, converged)

[1. 1.] 8.507945459257371e-10 True


## Problem 2

In [31]:
def smw(df,x0,maxiters):
# Sherman-Morrison-Woodbury Hessian-free optimization    
    
    n = np.size(x0)
    err = 100
    tol = 1e-9
    counter = 0
    Akinv = np.eye(n)
    xk = x0.copy()
    terminated, converged = False, False
    
    while (err > tol) and (counter+1 < maxiters ):       
        # Update x
        Dfxk = df(xk)
        xkp1 = xk - Akinv @ Dfxk.T
        
        # Update A_k
        sk = xkp1 - xk
        Dfxkp1 = df(xkp1)
        yk = Dfxkp1.T - Dfxk.T
        skdotyk = sk.T @ yk        
        term1den = (skdotyk) ** 2
        
        if term1den < 1e-50:
            print('Terminated because dviding by 0.')
            terminated = True
            converged = False
            break
        
        term1num = (skdotyk + yk.T @ Akinv @ yk) * np.outer(sk, sk)        
        term2num = Akinv @ np.outer(yk, sk) + np.outer(sk, yk) @ Akinv
        term2den = skdotyk
        
        Akp1inv = Akinv + term1num/term1den - term2num/term2den           
        
        err = la.norm(Dfxk, np.inf)
        xk = xkp1
        Akinv = Akp1inv
        counter +=1
    if counter < maxiters:
        converged = True
    return x, err, converged, terminated

In [32]:
x0 = np.array([-2,2])

x, err, converged, terminated = smw(df,x0,500)
print(x, err, converged, terminated)

[1. 1.] 9.115153076811434e-10 True False
